In [ ]:
from darts.models import ExponentialSmoothing
from darts.utils.utils import ModelMode, SeasonalityMode
from darts import TimeSeries
import pandas as pd
from pyprojroot import here

In [ ]:
origin = 8

In [ ]:
data = pd.read_csv(here() / 'data/preprocessed/targets.csv', parse_dates=True, index_col='Datetime')

In [ ]:
mask = data.columns.str.startswith('Target-Edor')

In [ ]:
edors = data.loc[:,mask]

In [ ]:
edors = edors.iloc[8:]

In [ ]:
forecasts = list()

for c in edors.columns:
    y = edors.loc[:,c]
    y = TimeSeries.from_series(y)
    model = ExponentialSmoothing(trend=ModelMode.ADDITIVE, 
                                seasonal=SeasonalityMode.ADDITIVE,
                                seasonal_periods=24)
    
    y_pred = model.historical_forecasts(y, 
                                        stride=24, 
                                        forecast_horizon=24, 
                                        verbose=True, 
                                        train_length=24*7,
                                        last_points_only=False)
    y_pred = pd.concat([x.pd_series() for x in y_pred])
    y_pred.name = c.replace('Target','Forecast_AHWM')
    forecasts.append(y_pred)

In [ ]:
ets = pd.concat(forecasts, axis=1)

In [ ]:
ets.index.name = 'Datetime'

In [ ]:
ets.to_csv(here() / 'data/preprocessed/ets.csv')